In [1]:
# This code has been produced by the group "team_pk" group, composed of Leonardo Suriano, Riccardo Pugliese and Mariana Dos Campos.
# In the code we have provided some comments, that aims to help the reader moving around the code and get what the code is doing.
# The whole explanation has been given for each step of the code, from the building features code to the final predictive model. 
# Moreover, we decided to import libraries not all at once, but to import in every cell the libraries that the cell is using. This choice 
# has been made in order to make clear which library has been used in that specific cell.
# In case the comments we added are not enough to satisfy your curiosity, and in case you may need further clarification about function
# taken from libraries, please refer to the documentation of the respective libraries.
# In case you need further clarification about function we created from scratch in our code or about how the libraries functions has
# been used, please feel free to contact us. We will be more than happy to answer all your doubt!!!




# AI assistance disclaimer
# Parts of this code (in particular some comments, the iterative feature search, and minor implementation details) may have been drafted or refined 
# with the help of AI-based tools. The use of AI was strictly limited to these aspects. All core ideas, modeling choices, and logical structures 
# implemented in the code and in the models were entirely conceived and designed by the members of the group, without external intellectual 
# contribution, relying solely on online documentation, our own knowledge and the insights provided by the course lectures.




# ============================================================
# In this first cell we load raw data and build features
# ------------------------------------------------------------
# In this cell we:
#   - load the raw battle logs from train.jsonl and test.jsonl
#   - define Pokémon base stats (species) and typing
#   - define a simple type chart (which type is strong/weak vs which)
#   - define helper functions for:
#       * safe division
#       * type effectiveness scoring
#       * simple entropy and run-length calculations
#   - define `create_features(data)` which loops over every battle
#     and builds a big dictionary of numeric features for that battle.
# It finally builds:
#   - `train_df`: one row per battle + the target `player_won`
#   - `test_df` : one row per battle (no target, we must predict it)
# ============================================================

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
import json
from collections import Counter

# ------------------------------------------------------------
# Paths for data files (train and test battle logs)
# ------------------------------------------------------------
DATA_DIR   = Path(r"C:\Users\2003l\OneDrive\Documenti\fds-pokemon-battles-prediction-2025")
TRAIN_FILE = DATA_DIR / "train.jsonl"
TEST_FILE  = DATA_DIR / "test.jsonl"

# Helper function to load a .jsonl file
# Each line is a separate JSON object (one battle per line).
def load_jsonl(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return [json.loads(line) for line in f if line.strip()]

print(f"Loading:\n- TRAIN: {TRAIN_FILE}\n- TEST : {TEST_FILE}")
train_data = load_jsonl(TRAIN_FILE)
test_data  = load_jsonl(TEST_FILE)
print(f"Train battles: {len(train_data)} | Test battles: {len(test_data)}")

# ------------------------------------------------------------
# Below, we are creating a dictionary of base stats for each Pokémon in our battles.
# For each species we store:
#   hp  = base HP
#   atk = attack
#   def = defense
#   spa = special attack
#   spd = special defense
#   spe = speed
# We will use these numbers to compute the features that need "edges",
# like speed advantage, attack advantage, defense advantage, etc.
# ------------------------------------------------------------

species = {
    'alakazam': {'hp': 55, 'atk': 50, 'def': 45, 'spa': 135, 'spd': 135, 'spe': 120},
    'articuno': {'hp': 90, 'atk': 85, 'def': 100, 'spa': 125, 'spd': 125, 'spe': 85},
    'chansey': {'hp': 250,'atk': 5, 'def': 5,  'spa': 105, 'spd': 105, 'spe': 50},
    'charizard':{'hp': 78, 'atk': 84, 'def': 78, 'spa': 85,  'spd': 85,  'spe': 100},
    'cloyster': {'hp': 50, 'atk': 95, 'def': 180,'spa': 85,  'spd': 85,  'spe': 70},
    'dragonite':{'hp': 91, 'atk': 134,'def': 95, 'spa': 100, 'spd': 100, 'spe': 80},
    'exeggutor':{'hp': 95, 'atk': 95, 'def': 85, 'spa': 125, 'spd': 125, 'spe': 55},
    'gengar':   {'hp': 60, 'atk': 65, 'def': 60, 'spa': 130, 'spd': 130, 'spe': 110},
    'golem':    {'hp': 80, 'atk': 110,'def': 130,'spa': 55,  'spd': 55,  'spe': 45},
    'jolteon':  {'hp': 65, 'atk': 65, 'def': 60, 'spa': 110, 'spd': 110, 'spe': 130},
    'jynx':     {'hp': 65, 'atk': 50, 'def': 35, 'spa': 95,  'spd': 95,  'spe': 95},
    'lapras':   {'hp': 130,'atk': 85, 'def': 80, 'spa': 95,  'spd': 95,  'spe': 60},
    'persian':  {'hp': 65, 'atk': 70, 'def': 60, 'spa': 65,  'spd': 65,  'spe': 115},
    'rhydon':   {'hp': 105,'atk': 130,'def': 120,'spa': 45,  'spd': 45,  'spe': 40},
    'slowbro':  {'hp': 95, 'atk': 75, 'def': 110,'spa': 80,  'spd': 80,  'spe': 30},
    'snorlax':  {'hp': 160,'atk': 110,'def': 65, 'spa': 65,  'spd': 65,  'spe': 30},
    'starmie':  {'hp': 60, 'atk': 75, 'def': 85, 'spa': 100, 'spd': 100, 'spe': 115},
    'tauros':   {'hp': 75, 'atk': 100,'def': 95, 'spa': 70,  'spd': 70,  'spe': 110},
    'victreebel':{'hp': 80,'atk': 105,'def': 65, 'spa': 100, 'spd': 100, 'spe': 70},
    'zapdos':   {'hp': 90, 'atk': 90, 'def': 85, 'spa': 125, 'spd': 125, 'spe': 100},
}

# ------------------------------------------------------------
# Typing for each Pokémon (type1, type2).
# "notype" means that slot is empty.
# ------------------------------------------------------------
types = {
    "alakazam":["notype","psychic"], "articuno":["flying","ice"], "chansey":["normal","notype"],
    "charizard":["fire","flying"], "cloyster":["ice","water"], "dragonite":["dragon","flying"],
    "exeggutor":["grass","psychic"], "gengar":["ghost","poison"], "golem":["ground","rock"],
    "jolteon":["electric","notype"], "jynx":["ice","psychic"], "lapras":["ice","water"],
    "persian":["normal","notype"], "rhydon":["ground","rock"], "slowbro":["psychic","water"],
    "snorlax":["normal","notype"], "starmie":["psychic","water"], "tauros":["normal","notype"],
    "victreebel":["grass","poison"], "zapdos":["electric","flying"]
}

# ------------------------------------------------------------
# Type chart: this tells us how strong one attacking type is
# against another defending type. This is due to the fact that 
# pokemon of one specific type may be really strong on some 
# specific type of pokemon but ineffective on others.
# An example has been provided:
#   effectiveness['fire']['grass'] = 2   (super effective)
#   effectiveness['normal']['rock'] = 0.5 (not very effective)
#   effectiveness['ghost']['psychic'] = 0 (no effect)
# ------------------------------------------------------------

# The following table may have been taken from external source, formatted as below and 
# copy pasted here.

effectiveness = {
    'normal': {'rock':0.5,'ghost':0,'notype':1},
    'fire':{'grass':2,'ice':2,'bug':2,'rock':0.5,'fire':0.5,'water':0.5,'dragon':0.5},
    'water':{'fire':2,'rock':2,'ground':2,'water':0.5,'grass':0.5,'dragon':0.5},
    'electric':{'water':2,'flying':2,'ground':0,'electric':0.5,'grass':0.5,'dragon':0.5},
    'grass':{'water':2,'rock':2,'ground':2,'fire':0.5,'grass':0.5,'poison':0.5,'flying':0.5,'dragon':0.5},
    'ice':{'grass':2,'ground':2,'flying':2,'dragon':2,'fire':0.5,'ice':0.5,'water':0.5},
    'poison':{'grass':2,'poison':0.5,'ground':0.5,'rock':0.5,'ghost':0.5},
    'ground':{'fire':2,'electric':2,'poison':2,'rock':2,'grass':0.5,'flying':0},
    'flying':{'grass':2,'fighting':2,'bug':2,'rock':0.5,'electric':0.5},
    'psychic':{'poison':2,'fighting':2,'psychic':0.5},
    'bug':{'grass':2,'psychic':2,'poison':0.5,'fire':0.5,'flying':0.5},
    'rock':{'fire':2,'ice':2,'flying':2,'bug':2,'ground':0.5},
    'ghost':{'ghost':2,'psychic':0},
    'dragon':{'dragon':2},
    'notype':{}
}

# ------------------------------------------------------------
# Convert the raw type chart into a simple integer "edge":
#   +1  = super effective
#   -1  = not very effective
#   -2  = no effect (immune)
#    0  = neutral
# This is easier to sum and interpret as a "type advantage".
# ------------------------------------------------------------
def type_match(tp1, tp2):
    v = effectiveness.get(tp1, {}).get(tp2, 1)
    if v == 0: return -2
    if v < 1:  return -1
    if v > 1:  return 1
    return 0

# ------------------------------------------------------------
# Map statuses to small integers representing their severity:
#   nostatus -> 0 (healthy)
#   par      -> 1 (mild but annoying)
#   brn      -> 1
#   psn      -> 1
#   tox      -> 2 (stronger poison)
#   frz      -> 2 (frozen)
#   slp      -> 3 (sleep, very severe here)
# We will sum/average these to measure "how bad" the statuses are.
# ------------------------------------------------------------
MAP_STATUS = {'nostatus':0,'par':1,'brn':1,'psn':1,'tox':2,'frz':2,'slp':3}

# Safe division: if denominator is zero, return 0.0 instead of crashing.
def _safe_div(a,b): 
    return float(a)/float(b) if float(b)!=0 else 0.0



# ------------------------------------------------------------
# Entropy helper: measure how "spread" a Counter is.
# High entropy = many mons used with similar frequency.
# Low entropy  = few mons used a lot.
# ------------------------------------------------------------
def _entropy(counter: Counter):
    tot = float(sum(counter.values()))
    if tot <= 0: return 0.0
    p = [c/tot for c in counter.values()]
    return float(-sum(pi*np.log(max(pi,1e-12)) for pi in p))

# Run-length helper: given a sequence like [A,A,A,B,B,C],
# we get [3,2,1] = lengths of consecutive runs.
def _run_lengths(seq):
    if not seq: return []
    runs, cur, cnt = [], seq[0], 1
    for s in seq[1:]:
        if s == cur: cnt += 1
        else: runs.append(cnt); cur, cnt = s, 1
    runs.append(cnt)
    return runs


# ============================================================
# FEATURE EXTRACTION
# ============================================================
def create_features(data: list[dict]) -> pd.DataFrame:
    """
    Turn a list of raw battles (from the JSONL files) into a clean table.
    
    For each battle we build a dictionary `f` with many numeric features that
    describe the match, for example:
      - HP gaps in different phases of the game (early / mid / overall)
      - damage dealt and healed by each player
      - status durations (paralysis, burn, poison, toxic, freeze, sleep)
      - type matchups and move power / accuracy statistics
      - switches, run lengths, initiative, KOs, and more.
    
    At the end we return a pandas DataFrame with:
      - one row per battle,
      - one column per feature,
    ready to be used as input for a machine learning model.
    """
    out = []  # this will store one dict 'f' per battle

    # Loop over all battles in the raw data
    for battle in tqdm(data, desc="Extracting features"):
        f = {}  # dictionary of features for the current battle

        # ---------------------------------------
        # TEAM-LEVEL FEATURES (PLAYER 1)
        # ---------------------------------------
        p1team = battle.get("p1_team_details") or []

        # Average Defense / Attack / Speed of P1's team (over all 6 mons).
        # These are "on-paper" stats, independent of the timeline.
        # They are used as baselines for some edges (e.g. atk_edge_used).
        p1_mean_def = np.mean([int(x.get("base_def", 0)) for x in p1team]) if p1team else 0.0
        p1_mean_atk = np.mean([int(x.get("base_atk", 0)) for x in p1team]) if p1team else 0.0
        p1_mean_spe = np.mean([int(x.get("base_spe", 0)) for x in p1team]) if p1team else 0.0

        # p1_mean_def: average base Defense of player 1's team.
        f["p1_mean_def"] = float(p1_mean_def)

        # Full timeline of the battle: list of turns
        tl = battle.get("battle_timeline") or []
        den = len(tl)  # number of turns in this battle


        # ---------------------------------------
        # LEAD POKÉMON FEATURES
        # ---------------------------------------
        lead_p1 = str(tl[0]["p1_pokemon_state"]["name"]).lower()
        lead_p2 = str(tl[0]["p2_pokemon_state"]["name"]).lower()

        # Defensive edge at lead: P2_def - P1_def.
        # Positive -> P2 lead is bulkier on paper; negative -> P1 bulkier.
        def_p1 = species.get(lead_p1, {}).get("def", 0)
        def_p2 = species.get(lead_p2, {}).get("def", 0)
        f["lead_def_edge"] = float(def_p2 - def_p1)  # lead Defense edge (p2 - p1)

        # Type edge at lead, using type_match:
        # We sum advantages of P2 attacking P1 and subtract advantages of P1 attacking P2.
        lt_p1 = [t for t in types.get(lead_p1, ["notype", "notype"]) if t]
        lt_p2 = [t for t in types.get(lead_p2, ["notype", "notype"]) if t]
        lead_type_edge = 0
        for t2 in lt_p2:
            for t1 in lt_p1:
                lead_type_edge += type_match(t2, t1)  # p2 hitting p1
                lead_type_edge -= type_match(t1, t2)  # p1 hitting p2
        f["lead_type_edge"] = int(lead_type_edge)  # positive -> type-favored lead for p2

        # Speed edge at lead: P2_speed - P1_speed.
        # Positive -> P2's lead is faster; negative -> P1's lead is faster.
        spe_p1 = species.get(lead_p1, {}).get("spe", 0)
        spe_p2 = species.get(lead_p2, {}).get("spe", 0)
        lead_speed_edge = spe_p2 - spe_p1  # used later in lead_speed_fb_agree

        # ---------------------------------------
        # BASIC TIMELINE ARRAYS
        # ---------------------------------------
        # HP% for each turn for both players
        p1_hp = [float(t["p1_pokemon_state"]["hp_pct"]) for t in tl]
        p2_hp = [float(t["p2_pokemon_state"]["hp_pct"]) for t in tl]

        # Raw status strings turn-by-turn (par, brn, slp, fnt, etc.)
        p1_stat_raw = [t["p1_pokemon_state"].get("status", "nostatus") for t in tl]
        p2_stat_raw = [t["p2_pokemon_state"].get("status", "nostatus") for t in tl]

        # Move details (type, power, accuracy...) for each turn
        p1_moves = [t.get("p1_move_details") or {} for t in tl]
        p2_moves = [t.get("p2_move_details") or {} for t in tl]

        # Which Pokémon is active on the field for each player at each turn
        p1_active = [str(t["p1_pokemon_state"]["name"]).lower() for t in tl]
        p2_active = [str(t["p2_pokemon_state"]["name"]).lower() for t in tl]

        # ---------------------------------------
        # SWITCHES AND RUN LENGTHS
        # ---------------------------------------
        # Total number of times each player switches to a different mon.
        # (Kept for possible debugging / interpretation but not stored as features.)
        p1_switches = sum(1 for a, b in zip(p1_active, p1_active[1:]) if a != b)
        p2_switches = sum(1 for a, b in zip(p2_active, p2_active[1:]) if a != b)

        # Last turn index (1-based) where P1 switched.
        # If P1 never switches, it stays at 1 (meaning "no switch, virtual early last switch").
        last_sw_p1 = 1
        c = 1
        for a, b in zip(p1_active, p1_active[1:]):
            c += 1
            if a != b:
                last_sw_p1 = c
        f["last_switch_turn_p1"] = int(last_sw_p1)  # last p1 switch turn (1-based)

        # Run lengths: how long each active streak lasts for both players.
        rl1 = _run_lengths(p1_active)
        rl2 = _run_lengths(p2_active)

        # p2_run_len_mean: average length (in turns) that P2 keeps the same mon active.
        f["p2_run_len_mean"] = float(np.mean(rl2)) if rl2 else 0.0

        # run_len_mean_diff: average run length difference (P2 - P1).
        f["run_len_mean_diff"] = float(
            (np.mean(rl2) if rl2 else 0.0) - (np.mean(rl1) if rl1 else 0.0)
        )

        # ---------------------------------------
        # HP-BASED FEATURES: DAMAGE, HEALING, GAP
        # ---------------------------------------
        # Split the battle into 3 chunks: early, mid, late.
        E = max(1, den // 3)
        M_end = max(E * 2, min(den, E * 2))  # approximate end of mid-game

        # Damage taken by each player per turn (HP drop)
        p1_loss = [max(0.0, p1_hp[i-1] - p1_hp[i]) for i in range(1, den)]
        p2_loss = [max(0.0, p2_hp[i-1] - p2_hp[i]) for i in range(1, den)]

        # p2_early_damage: total damage P2 takes in the early segment (first third).
        f["p2_early_damage"] = float(sum(p2_loss[:E]))

        # p2_late_damage: total damage P2 takes in the final segment (last third).
        f["p2_late_damage"] = float(sum(p2_loss[-E:]))

        # p1_mid_damage: total damage P1 takes in the mid segment.
        f["p1_mid_damage"] = float(sum(p1_loss[E:M_end]))

        # early_damage_gap: (damage P2 takes early) - (damage P1 takes early).
        # > 0 -> P2 is getting hit harder than P1 in early game.
        f["early_damage_gap"] = float(sum(p2_loss[:E]) - sum(p1_loss[:E]))

        # Healing (HP gains) for each player per turn
        p1_heal = [max(0.0, p1_hp[i] - p1_hp[i-1]) for i in range(1, den)]
        p2_heal = [max(0.0, p2_hp[i] - p2_hp[i-1]) for i in range(1, den)]

        # heal_mid_diff: total healing in mid game, P2 minus P1.
        f["heal_mid_diff"] = float(sum(p2_heal[E:M_end]) - sum(p1_heal[E:M_end]))

        # heal_late_diff: total healing in late game, P2 minus P1.
        f["heal_late_diff"] = float(sum(p2_heal[-E:]) - sum(p1_heal[-E:]))

        # HP gap at each turn: P2_HP - P1_HP.
        # > 0 -> P2 is ahead in HP; < 0 -> P1 is ahead.
        hp_gap = np.array(p2_hp) - np.array(p1_hp)

        # hp_gap_early: average HP gap in the early part.
        f["hp_gap_early"] = float(np.mean(hp_gap[:E])) if len(hp_gap) > 0 else 0.0

        # hp_gap_mid: average HP gap in the mid part.
        f["hp_gap_mid"] = float(np.mean(hp_gap[E:M_end])) if len(hp_gap) > E else 0.0

        # hp_gap_var: variance of HP gap across the whole match.
        f["hp_gap_var"] = float(np.var(hp_gap)) if len(hp_gap) > 1 else 0.0

        # hp_gap_sign_flips: how many times the sign of the HP gap changes
        # (excluding zeros) -> how often the lead changes between players.
        signs = np.sign(hp_gap)
        f["hp_gap_sign_flips"] = int(
            sum(1 for a, b in zip(signs, signs[1:]) if a != 0 and b != 0 and a != b)
        )

        # ---------------------------------------
        # "FIRST BLOOD" FEATURES (FIRST K.O.)
        # ---------------------------------------
        # Helper: find the first index where the opponent's mon faints (status goes to 'fnt').
        def _fb(prev_seq):
            return next(
                (i + 1 for i, (a, b) in enumerate(zip(prev_seq, prev_seq[1:]))
                 if a != 'fnt' and b == 'fnt'),
                None
            )

        # fb_p1: first turn where P2's mon goes from not fainted to fainted (P1 scores a KO).
        # fb_p2: first turn where P1's mon goes from not fainted to fainted (P2 scores a KO).
        fb_p1 = _fb(p2_stat_raw)  # first time P1 KOs a P2 mon
        fb_p2 = _fb(p1_stat_raw)  # first time P2 KOs a P1 mon
        firsts = [x for x in (fb_p1, fb_p2) if x is not None]
        fb_turn = min(firsts) if firsts else None

        # first_blood_happened: 1 if someone scored at least one KO, else 0.
        f["first_blood_happened"] = int(fb_turn is not None)

        # first_blood_side:
        #   1  -> P1 scored first KO (or ties resolved in favor of P1)
        #  -1  -> P2 scored first KO
        #   0  -> nobody fainted.
        f["first_blood_side"] = (
            1 if (fb_p1 is not None and (fb_p2 is None or fb_p1 <= fb_p2))
            else (-1 if fb_p2 is not None else 0)
        )

        # lead_type_fb_agree:
        #   1 if the sign of lead_type_edge and first_blood_side agree,
        #   i.e. the side favored by type at lead also gets first blood.
        f["lead_type_fb_agree"] = int(
            f["first_blood_side"] != 0 and np.sign(lead_type_edge) == f["first_blood_side"]
        )

        # lead_speed_fb_agree:
        #   1 if the sign of lead_speed_edge and first_blood_side agree,
        #   i.e. the faster lead side gets first blood.
        f["lead_speed_fb_agree"] = int(
            f["first_blood_side"] != 0 and np.sign(lead_speed_edge) == f["first_blood_side"]
        )

        # ---------------------------------------
        # STATUS FEATURES: COUNTS AND SEVERITY
        # ---------------------------------------
        # Convert raw status to numeric severity per turn using MAP_STATUS.
        p1_series = [MAP_STATUS.get(s, 0) for s in p1_stat_raw]
        p2_series = [MAP_STATUS.get(s, 0) for s in p2_stat_raw]

        # Counts of specific statuses (per turn) for each player.
        p1_par = sum(1 for s in p1_stat_raw if s == 'par')
        p2_par = sum(1 for s in p2_stat_raw if s == 'par')
        p1_frz = sum(1 for s in p1_stat_raw if s == 'frz')
        p1_brn = sum(1 for s in p1_stat_raw if s == 'brn')
        p2_brn = sum(1 for s in p2_stat_raw if s == 'brn')
        p1_psx = sum(1 for s in p1_stat_raw if s in {'psn', 'tox'})
        p2_psx = sum(1 for s in p2_stat_raw if s in {'psn', 'tox'})
        p2_slp = sum(1 for s in p2_stat_raw if s == 'slp')

        # p1_turns_par / p1_turns_frz / p1_turns_brn / p1_turns_psn_tox:
        #   total number of turns P1 spends under each specific condition.
        f["p1_turns_par"] = int(p1_par)
        f["p1_turns_frz"] = int(p1_frz)
        f["p1_turns_brn"] = int(p1_brn)
        f["p1_turns_psn_tox"] = int(p1_psx)

        # p2_turns_brn / p2_turns_psn_tox / p2_turns_slp:
        #   total number of turns P2 spends under each specific condition.
        f["p2_turns_brn"] = int(p2_brn)
        f["p2_turns_psn_tox"] = int(p2_psx)
        f["p2_turns_slp"] = int(p2_slp)

        # par_turns_diff: difference in paralysis turns (P1 - P2).
        f["par_turns_diff"] = int(p1_par - p2_par)

        # severe2_turns_diff:
        #   difference in very severe statuses (freeze + sleep):
        #   (P1 freeze + P1 sleep) - (P2 freeze + P2 sleep).
        f["severe2_turns_diff"] = int(
            (p1_frz + sum(1 for s in p1_stat_raw if s == 'slp')) -
            (p2_slp + sum(1 for s in p2_stat_raw if s == 'frz'))
        )

        # severe_mask: turns where at least one side has severity >= 2
        # (e.g. tox, frz, slp depending on MAP_STATUS).
        severe_mask = [(a >= 2 or b >= 2) for a, b in zip(p1_series, p2_series)]

        # severe_status_share: fraction of turns (overall) with at least one severe status.
        f["severe_status_share"] = _safe_div(
            sum(1 for x in severe_mask if x), den
        )

        # severe_status_early_share: fraction of early turns with a severe status.
        f["severe_status_early_share"] = _safe_div(
            sum(1 for x in severe_mask[:E] if x), max(1, E)
        )

        # ---------------------------------------
        # "REVEALED" AND DIVERSITY FEATURES
        # ---------------------------------------
        # Which species have actually appeared on the field?
        p1_seen = set(p1_active)
        p2_seen = set(p2_active)

        # revealed_count_diff:
        #   (# unique mons used by P1) - (# unique mons used by P2).
        f["revealed_count_diff"] = int(len(p1_seen) - len(p2_seen))

        # p2_switch_early_share:
        #   proportion of early transitions where P2 changes Pokémon.
        f["p2_switch_early_share"] = _safe_div(
            sum(1 for a, b in zip(p2_active[:E], p2_active[1:E]) if a != b),
            den
        )

        # Entropy of active mons: how much a player rotated between different species.
        c1 = Counter(p1_active)
        c2 = Counter(p2_active)
        p1_ent = _entropy(c1)
        p2_ent = _entropy(c2)

        # active_entropy_diff: diversity of usage difference = H(P1 active) - H(P2 active).
        f["active_entropy_diff"] = float(p1_ent - p2_ent)

        # ---------------------------------------
        # INITIATIVE / SPEED FEATURES
        # ---------------------------------------
        # Helper: is P1's active mon at least as fast as P2's at turn i (base speed only)?
        def _p1_faster(i):
            n1 = p1_active[i]
            n2 = p2_active[i]
            return species.get(n1, {}).get("spe", 0) >= species.get(n2, {}).get("spe", 0)

        # p1_fast: number of turns where P1 is at least as fast as P2.
        p1_fast = sum(1 for i in range(den) if _p1_faster(i))

        # initiative_early_diff:
        #   (share of early turns where P1 is faster/equal)
        #   - (share of early turns where P2 is faster).
        f["initiative_early_diff"] = (
            _safe_div(sum(1 for i in range(min(E, den)) if _p1_faster(i)), max(1, E))
            - _safe_div(sum(1 for i in range(min(E, den)) if not _p1_faster(i)), max(1, E))
        )

        # initiative_late_diff:
        #   same measure but computed in the last E turns.
        f["initiative_late_diff"] = (
            _safe_div(sum(1 for i in range(max(0, den-E), den) if _p1_faster(i)), max(1, E))
            - _safe_div(sum(1 for i in range(max(0, den-E), den) if not _p1_faster(i)), max(1, E))
        )

        # ---------------------------------------
        # USED MEAN SPEED / ATK FEATURES
        # ---------------------------------------
        # Weighted mean by how many turns each mon was active.
        def _wmean(counter, key):
            vals = [(species[n][key], w) for n, w in counter.items() if n in species]
            return float(sum(v * w for v, w in vals)) / float(sum(w for _, w in vals)) if vals else 0.0

        p1_used_mean_spe = _wmean(c1, 'spe')  # avg base speed of p1 mons, weighted by time on field
        p2_used_mean_spe = _wmean(c2, 'spe')  # avg base speed of p2 mons, weighted by time on field
        p2_used_mean_atk = _wmean(c2, 'atk')  # avg base attack of p2 mons, weighted by time on field

        # p1_used_mean_spe: weighted average Speed stat of P1's used mons.
        f["p1_used_mean_spe"] = float(p1_used_mean_spe)

        # used_mean_spe_diff: (p1_used_mean_spe) - (p2_used_mean_spe).
        f["used_mean_spe_diff"] = float(p1_used_mean_spe - p2_used_mean_spe)

        # p2_used_count: number of distinct mons P2 actually used.
        f["p2_used_count"] = int(len(c2))

        # atk_edge_used:
        #   (P2's weighted mean Attack of used mons) - (P1 team mean Attack).
        f["atk_edge_used"] = float(p2_used_mean_atk - p1_mean_atk)

        # ---------------------------------------
        # FINAL STATE FEATURES (END OF BATTLE)
        # ---------------------------------------
        # Track last HP% and last status seen for each mon of both players.
        last_hp_p1, last_hp_p2, last_st_p1, last_st_p2 = {}, {}, {}, {}
        for t in tl:
            n1 = str(t["p1_pokemon_state"]["name"]).lower()
            n2 = str(t["p2_pokemon_state"]["name"]).lower()
            last_hp_p1[n1] = float(t["p1_pokemon_state"]["hp_pct"])
            last_hp_p2[n2] = float(t["p2_pokemon_state"]["hp_pct"])
            last_st_p1[n1] = t["p1_pokemon_state"].get("status", "nostatus")
            last_st_p2[n2] = t["p2_pokemon_state"].get("status", "nostatus")

        # Count mons each side still has alive at the end (>0 HP).
        p1_alive_final = sum(1 for v in last_hp_p1.values() if v > 0)
        p2_alive_final = sum(1 for v in last_hp_p2.values() if v > 0)

        # alive_diff_final:
        #   (alive mons for P1 at end) - (alive mons for P2 at end).
        f["alive_diff_final"] = int(p1_alive_final - p2_alive_final)

        # Mean HP% of all mons at the end (per side).
        mean_hp_p1 = float(np.mean(list(last_hp_p1.values()))) if last_hp_p1 else 0.0
        mean_hp_p2 = float(np.mean(list(last_hp_p2.values()))) if last_hp_p2 else 0.0

        # hp_edge_final: final HP edge = mean HP of P2 mons - mean HP of P1 mons.
        f["hp_edge_final"] = float(mean_hp_p2 - mean_hp_p1)

        # mean_remaining_hp_p2: mean P2 final HP% (directly).
        f["mean_remaining_hp_p2"] = float(mean_hp_p2)

        # Status severity at the end (average over mons) using MAP_STATUS.
        p1_status_mean_final = float(np.mean([MAP_STATUS.get(s, 0) for s in last_st_p1.values()])) if last_st_p1 else 0.0
        p2_status_mean_final = float(np.mean([MAP_STATUS.get(s, 0) for s in last_st_p2.values()])) if last_st_p2 else 0.0

        # p1_status_mean_final: mean status severity of P1's team at the end.
        f["p1_status_mean_final"] = float(p1_status_mean_final)

        # status_severity_gap_final:
        #   (mean final status severity for P2) - (mean final status severity for P1).
        f["status_severity_gap_final"] = float(p2_status_mean_final - p1_status_mean_final)

        # ---------------------------------------
        # TYPE EDGE DURING MOVES
        # ---------------------------------------
        # Average type-advantage score for moves used by each side.
        def _avg_edge(side_key, opp_key):
            vals = []
            for t in tl:
                md = t.get(side_key) or {}
                if not md.get("name"):
                    continue
                mt = str(md.get("type", "")).lower()
                if not mt:
                    continue
                oppn = str(t[opp_key]["name"]).lower()
                opp_types = [tp for tp in types.get(oppn, ["notype", "notype"]) if tp != 'notype']
                if not opp_types:
                    continue
                s = 0
                for otp in opp_types:
                    s += type_match(mt, otp)
                vals.append(s)
            return float(np.mean(vals)) if vals else 0.0

        # p1_to_p2: avg type advantage of P1's moves against P2's active mons.
        # p2_to_p1: avg type advantage of P2's moves against P1's active mons.
        p1_to_p2 = _avg_edge("p1_move_details", "p2_pokemon_state")
        p2_to_p1 = _avg_edge("p2_move_details", "p1_pokemon_state")

        # type_edge_avg_diff:
        #   (P1 attacking P2) - (P2 attacking P1).
        f["type_edge_avg_diff"] = float(p1_to_p2 - p2_to_p1)

        # p2_to_p1_type_edge_avg:
        #   average type advantage score of P2's moves vs P1.
        f["p2_to_p1_type_edge_avg"] = float(p2_to_p1)

        # ---------------------------------------
        # TYPE DIVERSITY FEATURES
        # ---------------------------------------
        # Types seen among all mons that appeared on each side.
        p1_seen_types = set()
        for n in p1_seen:
            for tp in types.get(n, []):
                if tp != 'notype':
                    p1_seen_types.add(tp)

        p2_seen_types = set()
        for n in p2_seen:
            for tp in types.get(n, []):
                if tp != 'notype':
                    p2_seen_types.add(tp)

        # p2_seen_type_count: how many distinct types P2 showed.
        f["p2_seen_type_count"] = int(len(p2_seen_types))

        # type_seen_count_diff: (# distinct types P1 showed) - (# distinct types P2 showed).
        f["type_seen_count_diff"] = int(len(p1_seen_types) - len(p2_seen_types))

        # ---------------------------------------
        # DAMAGE STATISTICS FOR P2
        # ---------------------------------------
        # p2_damage_std: std dev of P2's HP loss per turn.
        f["p2_damage_std"] = float(np.std(p2_loss)) if p2_loss else 0.0

        # p2_damage_median: median HP loss of P2 per turn.
        f["p2_damage_median"] = float(np.median(p2_loss)) if p2_loss else 0.0

        # ---------------------------------------
        # MOVE POWER / ACCURACY FEATURES
        # ---------------------------------------
        bp1 = [float(m.get("base_power")) for m in p1_moves if m.get("base_power") not in (None, "")]
        bp2 = [float(m.get("base_power")) for m in p2_moves if m.get("base_power") not in (None, "")]

        # bp_std_p1: standard deviation of P1 move base powers.
        f["bp_std_p1"] = float(np.std(bp1)) if bp1 else 0.0

        # bp_mean_p2: mean base power of P2's moves.
        f["bp_mean_p2"] = float(np.mean(bp2)) if bp2 else 0.0

        # bp_std_diff: difference in std dev of move power (P2 - P1).
        f["bp_std_diff"] = float(
            (np.std(bp2) if bp2 else 0.0) - (np.std(bp1) if bp1 else 0.0)
        )

        acc1 = [float(m.get("accuracy")) for m in p1_moves if m.get("accuracy") not in (None, "")]
        acc2 = [float(m.get("accuracy")) for m in p2_moves if m.get("accuracy") not in (None, "")]

        # acc_mean_p2: mean accuracy of P2's moves.
        f["acc_mean_p2"] = float(np.mean(acc2)) if acc2 else 0.0

        # acc_mean_diff: mean accuracy difference (P2 - P1).
        f["acc_mean_diff"] = float(
            (np.mean(acc2) if acc2 else 0.0) - (np.mean(acc1) if acc1 else 0.0)
        )

        # Count how many moves are actually used (move name not None).
        atk1 = sum(1 for m in p1_moves if (m.get("name") is not None))
        atk2 = sum(1 for m in p2_moves if (m.get("name") is not None))

        # low1 / low2: counts of low-accuracy moves (< 1.0) used by each side.
        low1 = sum(1 for m in p1_moves if m.get("accuracy") not in (None, "") and float(m["accuracy"]) < 1.0)
        low2 = sum(1 for m in p2_moves if m.get("accuracy") not in (None, "") and float(m["accuracy"]) < 1.0)

        # low_acc_share_diff:
        #   (fraction of low-accuracy moves used by P2) - (same fraction for P1).
        f["low_acc_share_diff"] = _safe_div(low2, max(1, atk2)) - _safe_div(low1, max(1, atk1))

        # attacks_rate_diff:
        #   (attacks per turn for P1) - (attacks per turn for P2).
        f["attacks_rate_diff"] = _safe_div(atk1, den) - _safe_div(atk2, den)

        # ---------------------------------------
        # TYPE-BASED RESIST / IMMUNE FEATURES
        # ---------------------------------------
        # Count how many times moves hit into resistances or immunities.
        def _edge_counts(side_key, opp_key):
            c_res, c_imm = 0, 0
            for t in tl:
                md = t.get(side_key) or {}
                if not md.get("name"):
                    continue
                mt = str(md.get("type", "")).lower()
                if not mt:
                    continue
                oppn = str(t[opp_key]["name"]).lower()
                opp_types = [tp for tp in types.get(oppn, ["notype", "notype"]) if tp != 'notype']
                for otp in opp_types:
                    v = type_match(mt, otp)
                    if v == -1:
                        c_res += 1   # not very effective
                    if v == -2:
                        c_imm += 1   # no effect
            return c_res, c_imm

        # p1_res / p1_imm: # of resisted / immune hits produced by P1's moves.
        # p2_res / p2_imm: # of resisted / immune hits produced by P2's moves.
        p1_res, p1_imm = _edge_counts("p1_move_details", "p2_pokemon_state")
        p2_res, p2_imm = _edge_counts("p2_move_details", "p1_pokemon_state")

        # resist_count_diff:
        #   (# resisted hits caused by P1) - (# resisted hits caused by P2).
        f["resist_count_diff"] = int(p1_res - p2_res)

        # p1_immune_count: how many times P1 hits into an immunity.
        f["p1_immune_count"] = int(p1_imm)

        # immune_count_diff:
        #   (# immunities triggered by P1's moves) - (# triggered by P2's moves).
        f["immune_count_diff"] = int(p1_imm - p2_imm)

        # ---------------------------------------
        # BOOST FEATURES
        # ---------------------------------------
        # Helper: sum of positive stat boosts in a dict.
        def _boost_sum_pos(d):
            return sum(max(0, int(v)) for v in (d or {}).values())

        # Count turns where each player has any positive boosts active.
        p1_boost_turns = sum(
            1 for t in tl if _boost_sum_pos(t["p1_pokemon_state"].get("boosts", {})) > 0
        )
        p2_boost_turns = sum(
            1 for t in tl if _boost_sum_pos(t["p2_pokemon_state"].get("boosts", {})) > 0
        )

        # boost_turns_diff:
        #   (# boost turns for P1) - (# boost turns for P2).
        f["boost_turns_diff"] = int(p1_boost_turns - p2_boost_turns)

        # p2_max_boost_sum:
        #   maximum sum of positive boosts that P2 reaches in any turn.
        f["p2_max_boost_sum"] = int(
            max((_boost_sum_pos(t["p2_pokemon_state"].get("boosts", {})) for t in tl), default=0)
        )

        # ---------------------------------------
        # LAST TURN TYPE EDGE (TYPES_LAST_ROUND)
        # ---------------------------------------
        last_p1 = str(tl[-1]["p1_pokemon_state"]["name"]).lower()
        last_p2 = str(tl[-1]["p2_pokemon_state"]["name"]).lower()
        t1 = [tp for tp in types.get(last_p1, ["notype", "notype"])]
        t2 = [tp for tp in types.get(last_p2, ["notype", "notype"])]
        tot = 0
        for a in t1:
            for b in t2:
                tot += type_match(a, b)  # p1 hitting p2
                tot -= type_match(b, a)  # p2 hitting p1

        # types_last_round:
        #   net type edge in the final board position (positive -> p1 favored).
        f["types_last_round"] = int(tot)

        # ---------------------------------------
        # KO RATE FEATURE
        # ---------------------------------------
        # p1_kos: how many times P2 transitions from not fainted to fainted.
        p1_kos = sum(1 for a, b in zip(p2_stat_raw, p2_stat_raw[1:]) if a != 'fnt' and b == 'fnt')

        # p2_kos: how many times P1 transitions from not fainted to fainted.
        p2_kos = sum(1 for a, b in zip(p1_stat_raw, p1_stat_raw[1:]) if a != 'fnt' and b == 'fnt')

        # ko_rate_total:
        #   average number of KOs per turn (both sides combined).
        f["ko_rate_total"] = _safe_div(p1_kos + p2_kos, den)

        # ---------------------------------------
        # LATE SWITCH SHARE FOR P1
        # ---------------------------------------
        Esize = E
        # p1_sw_late: number of switches P1 makes in the last E turns.
        p1_sw_late = (
            sum(1 for a, b in zip(p1_active[-Esize:], p1_active[-Esize+1:]) if a != b)
            if den > 1 else 0
        )

        # p1_switch_late_share:
        #   share of late turns (relative to all turns) where P1 switches.
        f["p1_switch_late_share"] = _safe_div(p1_sw_late, den)

        # ---------------------------------------
        # IDENTIFIERS AND TARGET
        # ---------------------------------------
        f["battle_id"] = battle.get("battle_id")
        if "player_won" in battle:
            f["player_won"] = int(battle["player_won"])

        out.append(f)

    # Turn the list of feature dicts into a DataFrame
    return pd.DataFrame(out).fillna(0)

# For an easy understanding of what these variables actually means, we have here a better description:


# ============================================================
# FEATURE GLOSSARY FOR create_features()
# ============================================================
#
# General conventions
# -------------------
# Many features are differences of the form:
#   - (P2 - P1)
#   - (P1 - P2)
# The sign always tells you who is advantaged:
#   - Positive (P2 - P1)  -> advantage for P2
#   - Positive (P1 - P2)  -> advantage for P1
#
# 1. TEAM-LEVEL FEATURES
# ----------------------
# p1_mean_def
#   Average base Defense stat of player 1’s team, computed over all 6 Pokémon
#   in p1_team_details. It is used as a defensive baseline for P1.
#
# 2. LEAD FEATURES
# ----------------
# lead_def_edge
#   Base Defense difference at lead:
#   (Defense of P2’s lead Pokémon) − (Defense of P1’s lead Pokémon).
#   > 0  -> on paper, P2’s lead is bulkier than P1’s lead.
#
# lead_type_edge
#   Type matchup edge at lead between lead_p2 and lead_p1.
#   It aggregates type_match scores:
#     - P2 attacking P1 (positive contributions)
#     - P1 attacking P2 (negative contributions).
#   > 0  -> type matchup at lead is favorable to P2.
#
# (lead_speed_edge is used internally but not stored as a feature.)
#
# 3. SWITCHES AND RUN LENGTHS
# ---------------------------
# last_switch_turn_p1
#   Last turn index (1-based) where P1 changes the active Pokémon.
#   If P1 never switches, it is set to 1 (a virtual early "last switch").
#
# p2_run_len_mean
#   Mean run length of P2’s active Pokémon: average number of consecutive turns
#   that P2 keeps the same mon on the field before switching.
#
# run_len_mean_diff
#   Difference in average run length:
#   (mean run length of P2’s active mons) − (mean run length of P1’s active mons).
#   > 0  -> P2 tends to stay longer with each mon than P1.
#
# 4. HP-BASED DAMAGE / HEALING / GAP
# ----------------------------------
# p2_early_damage
#   Total HP lost by P2 in the early third of the battle.
#
# p2_late_damage
#   Total HP lost by P2 in the final third of the battle.
#
# p1_mid_damage
#   Total HP lost by P1 in the middle third of the battle.
#
# early_damage_gap
#   (damage taken by P2 in the early game) − (damage taken by P1 in the early game).
#   > 0  -> P2 is taking more damage than P1 in early turns.
#
# heal_mid_diff
#   In the mid segment:
#   (total healing performed by P2) − (total healing performed by P1).
#   > 0  -> P2 heals more than P1 in the mid game.
#
# heal_late_diff
#   In the last third of the battle:
#   (total healing performed by P2) − (total healing performed by P1).
#   > 0  -> P2 heals more than P1 in the late game.
#
# hp_gap(t)
#   Defined internally as HP_p2(t) − HP_p1(t).
#   > 0  -> P2 is ahead in total HP at turn t.
#
# hp_gap_early
#   Mean HP gap in the early third:
#   average over turns of (HP_p2 − HP_p1) in the early segment.
#
# hp_gap_mid
#   Mean HP gap in the mid-game segment.
#
# hp_gap_var
#   Variance of HP gap across the whole battle.
#   Higher values indicate more volatile, swingy games.
#
# hp_gap_sign_flips
#   Number of times the sign of hp_gap changes between consecutive turns
#   (ignoring zeros). Rough measure of how often the lead changes between players.
#
# 5. FIRST BLOOD (FIRST K.O.) FEATURES
# ------------------------------------
# first_blood_happened
#   1 if at least one Pokémon fainted during the battle; 0 otherwise.
#
# first_blood_side
#   Encodes which side scored the first KO:
#     1  -> player 1 delivered the first KO (or tied but considered P1)
#    -1  -> player 2 delivered the first KO
#     0  -> no faint occurred in the battle.
#
# lead_type_fb_agree
#   1 if the sign of lead_type_edge and first_blood_side agree:
#   the side favored by the lead type matchup also gets first blood.
#   0 otherwise.
#
# lead_speed_fb_agree
#   1 if the sign of lead_speed_edge (P2_speed - P1_speed) and first_blood_side agree:
#   the side with the faster lead gets first blood.
#   0 otherwise.
#
# 6. STATUS COUNTS AND SEVERITY
# -----------------------------
# Status strings used:
#   'par' (paralyzed), 'brn' (burned), 'psn' (poisoned), 'tox' (badly poisoned),
#   'frz' (frozen), 'slp' (asleep), 'fnt' (fainted), 'nostatus' (healthy).
# Numerical severity is given by MAP_STATUS.
#
# p1_turns_par
#   Total number of turns where P1’s active Pokémon is paralyzed.
#
# p1_turns_frz
#   Total number of turns where P1’s active Pokémon is frozen.
#
# p1_turns_brn
#   Total number of turns where P1’s active Pokémon is burned.
#
# p1_turns_psn_tox
#   Total number of turns where P1’s active Pokémon is either poisoned or badly poisoned
#   (status in {psn, tox}).
#
# p2_turns_brn
#   Total number of turns where P2’s active Pokémon is burned.
#
# p2_turns_psn_tox
#   Total number of turns where P2’s active Pokémon is poisoned or badly poisoned.
#
# p2_turns_slp
#   Total number of turns where P2’s active Pokémon is asleep.
#
# par_turns_diff
#   Difference in paralysis duration:
#   (p1_turns_par) − (p2_turns_par).
#   > 0  -> P1 spends more turns paralyzed than P2.
#
# severe2_turns_diff
#   Difference in combined severe status (freeze + sleep):
#   (# freeze + sleep turns for P1) − (# freeze + sleep turns for P2).
#
# severe_status_share
#   Fraction of turns in the entire battle where at least one side has
#   a severe status (MAP_STATUS ≥ 2, e.g. toxic, freeze, sleep).
#
# severe_status_early_share
#   Fraction of early turns where at least one side has a severe status.
#
# 7. REVEALED / DIVERSITY FEATURES
# --------------------------------
# revealed_count_diff
#   Difference in number of distinct species that appeared on the field:
#   (# unique mons used by P1) − (# unique mons used by P2).
#
# p2_switch_early_share
#   Proportion of early transitions where P2 changes Pokémon.
#   It is computed as:
#     (# of times P2's active Pokémon changes in the early window) / total_turns.
#
# active_entropy_diff
#   Difference in entropy of active Pokémon usage:
#   H(P1 active sequence) − H(P2 active sequence).
#   High positive values mean P1 used a more diverse, balanced mix of mons.
#
# 8. INITIATIVE / SPEED FEATURES
# ------------------------------
# The helper notion "_p1_faster(i)" is true if P1's base Speed ≥ P2's base Speed at turn i.
#
# initiative_early_diff
#   In the early third of the battle:
#   (share of turns where P1 is at least as fast as P2)
#   − (share of turns where P2 is strictly faster than P1).
#   Positive values indicate an early speed advantage for P1.
#
# initiative_late_diff
#   Same construction as initiative_early_diff, but computed over the last
#   third of the battle (late game).
#
# 9. USED MEAN SPEED / ATK FEATURES
# ---------------------------------
# p1_used_mean_spe
#   Usage-weighted mean base Speed of P1's Pokémon, where the weight is
#   the number of turns each mon spends on the field.
#
# used_mean_spe_diff
#   Weighted Speed difference:
#   (p1_used_mean_spe) − (usage-weighted mean Speed of P2's mons).
#   > 0  -> on average, P1's active mons are faster than P2's.
#
# p2_used_count
#   Number of distinct Pokémon that P2 actually used (took the field at least once).
#
# atk_edge_used
#   Attack edge of P2’s used mons against P1’s team baseline:
#   (usage-weighted mean base Attack of P2’s used Pokémon)
#   − (mean base Attack of P1’s whole team).
#   > 0  -> P2’s used attackers are stronger on paper than P1’s team average.
#
# 10. FINAL STATE (END-OF-BATTLE) FEATURES
# ----------------------------------------
# alive_diff_final
#   Difference in number of Pokémon still alive at the end:
#   (# of P1 mons with HP > 0) − (# of P2 mons with HP > 0).
#   > 0  -> P1 has more surviving mons than P2.
#
# hp_edge_final
#   Final HP edge:
#   (mean final HP% of P2’s mons) − (mean final HP% of P1’s mons).
#   > 0  -> P2 ends the game with more HP on average.
#
# mean_remaining_hp_p2
#   Mean final HP% of P2’s mons alone (no difference).
#
# p1_status_mean_final
#   Mean final status severity of P1’s team (using MAP_STATUS).
#
# status_severity_gap_final
#   Final status severity difference:
#   (mean final severity for P2) − (mean final severity for P1).
#   > 0  -> P2’s team is, on average, in a worse status condition than P1's at the end.
#
# 11. TYPE EDGE DURING MOVES
# --------------------------
# type_edge_avg_diff
#   Average type advantage from moves:
#   (mean type_match score for P1 attacking P2) − (mean type_match for P2 attacking P1).
#   > 0  -> P1’s moves are, on average, more type-favored than P2’s.
#
# p2_to_p1_type_edge_avg
#   Mean type advantage score of P2’s moves against P1’s active Pokémon.
#   Higher values mean P2 is hitting into more favorable type matchups.
#
# 12. TYPE DIVERSITY FEATURES
# ---------------------------
# p2_seen_type_count
#   Number of distinct elemental types seen on P2’s side among all Pokémon that
#   actually appeared on the field.
#
# type_seen_count_diff
#   Difference in type diversity:
#   (# of distinct types seen on P1’s side) − (# of distinct types on P2’s side).
#
# 13. DAMAGE STATISTICS FOR P2
# ----------------------------
# p2_damage_std
#   Standard deviation of P2’s HP loss per turn.
#   High values indicate that some turns P2 takes much more damage than others.
#
# p2_damage_median
#   Median HP loss per turn for P2.
#
# 14. MOVE POWER / ACCURACY FEATURES
# ----------------------------------
# bp_std_p1
#   Standard deviation of base power of P1’s moves (over turns where base_power exists).
#
# bp_mean_p2
#   Mean base power of P2’s moves.
#
# bp_std_diff
#   (std dev of base power for P2’s moves) − (std dev for P1’s moves).
#
# acc_mean_p2
#   Mean accuracy of P2’s moves (ignoring moves with missing accuracy).
#
# acc_mean_diff
#   Difference in mean accuracy:
#   (mean accuracy of P2’s moves) − (mean accuracy of P1’s moves).
#
# low_acc_share_diff
#   Difference in the share of low-accuracy moves (< 1.0):
#   (fraction of P2’s moves with accuracy < 1.0) − (fraction for P1).
#
# attacks_rate_diff
#   Attack frequency difference:
#   (attacks per turn for P1) − (attacks per turn for P2),
#   where an “attack” is any turn with a non-empty move name.
#
# 15. TYPE-BASED RESIST / IMMUNE FEATURES
# ---------------------------------------
# resist_count_diff
#   Difference in how often each side hits into resistances:
#   (# of resisted hits caused by P1’s moves) − (# caused by P2’s moves).
#
# p1_immune_count
#   Number of times P1’s moves hit into immunities (type_match == -2).
#
# immune_count_diff
#   (# of immunities triggered by P1’s moves) − (# of immunities triggered by P2’s moves).
#
# 16. BOOST FEATURES
# ------------------
# boost_turns_diff
#   Difference in how many turns each side has any positive stat boost:
#   (# of turns P1 has boosts > 0) − (# of turns P2 has boosts > 0).
#
# p2_max_boost_sum
#   Maximum sum of positive stat boosts for P2 over all turns:
#   for each turn, sum all positive stage boosts in P2’s boosts dict
#   and take the maximum across the battle.
#
# 17. LAST TURN TYPE EDGE
# -----------------------
# types_last_round
#   Type matchup edge on the final board:
#   sum of type_match scores of P1’s active mon attacking P2’s active mon
#   minus the reverse (P2 attacking P1).
#   > 0  -> final position is type-favored for P1.
#
# 18. KO RATE
# -----------
# ko_rate_total
#   Average number of KOs per turn:
#   (total KOs dealt by P1 + total KOs dealt by P2) / number_of_turns.
#
# 19. LATE SWITCH SHARE FOR P1
# ----------------------------
# p1_switch_late_share
#   In the last E turns (late game), this measures how often P1 switches,
#   normalized by total number of turns:
#   (# of P1 switches in last E turns) / total_turns.
#
# 20. IDENTIFIERS / TARGET
# ------------------------
# battle_id
#   Unique identifier of the battle, as read from the JSONL file.
#
# player_won
#   Target variable (only in train): 1 if player 1 won the battle, 0 otherwise.
#
# ============================================================
# End of feature glossary for create_features()
# ============================================================







# =========================
# Our code is basically creating two dataset, a train and a test one, based on the train and test sets provided by the teachers. 
# In the train set there are 68 columns (features), that are: "hp_edge_final","revealed_count_diff","status_severity_gap_final","ko_rate_total",
# "severe2_turns_diff","attacks_rate_diff","active_entropy_diff","status_diversity_diff",
# "p2_used_count","par_turns_diff","type_edge_avg_diff","p2_late_damage","used_mean_spe_diff",
# "p1_status_mean_final","alive_diff_final","types_last_round","lead_type_edge","last_switch_turn_p1",
# "atk_edge_used","p1_turns_psn_tox","severe_status_early_share","p2_turns_psn_tox","initiative_early_diff",
# "bp_mean_p2","initiative_late_diff","first_blood_happened","run_len_mean_diff","heal_late_diff",
# "lead_def_edge","p2_max_boost_sum","p2_damage_std","p1_switch_late_share","p1_immune_count",
# "immune_count_diff","early_damage_gap","first_blood_side","severe_status_share","p2_turns_brn","hp_gap_mid",
# "hp_gap_var","bp_std_diff","hp_gap_early","p2_damage_median","p2_to_p1_type_edge_avg","p2_seen_type_count",
# "p1_turns_frz","low_acc_share_diff","acc_mean_diff","p2_run_len_mean","type_seen_count_diff",
# "mean_remaining_hp_p2","p1_mid_damage","p1_turns_par","boost_turns_diff","resist_count_diff",
# "p1_turns_brn","p1_mean_def","p2_switch_early_share","lead_type_fb_agree","hp_gap_sign_flips",
# "p1_used_mean_spe","p2_turns_slp","heal_mid_diff","p2_early_damage","bp_std_p1","acc_mean_p2",
# "lead_speed_fb_agree"



# ------------------------------------------------------------
# Build train_df and test_df
# ------------------------------------------------------------
train_df = create_features(train_data)
test_df  = create_features(test_data)
print(f"[FINAL] train_df: {train_df.shape}")
print(f"[FINAL] test_df : {test_df.shape}")


Loading:
- TRAIN: C:\Users\2003l\OneDrive\Documenti\fds-pokemon-battles-prediction-2025\train.jsonl
- TEST : C:\Users\2003l\OneDrive\Documenti\fds-pokemon-battles-prediction-2025\test.jsonl
Train battles: 10000 | Test battles: 5000


Extracting features:   0%|          | 0/10000 [00:00<?, ?it/s]

Extracting features:   0%|          | 0/5000 [00:00<?, ?it/s]

[FINAL] train_df: (10000, 68)
[FINAL] test_df : (5000, 67)


In [2]:
# In this second cell, we build X, y, X_test (NUMPY ARRAYS)
# ------------------------------------------------------------
# At this point we already have two DataFrames:
#   - `train_df`: one row per battle, features + `player_won` + `battle_id`
#   - `test_df` : one row per battle, the same features + `battle_id`
#
# In this cell we:
#   1. Find the columns that are present in BOTH train_df and test_df.
#   2. Drop the identifier/target columns ('battle_id', 'player_won') from the feature list.
#   3. Build three numpy arrays:
#        - X      : training features
#        - y      : training labels (0/1, did player 1 win?)
#        - X_test : test features (we will predict y for these rows)
# ============================================================

# In this cell we have taken our variables, we have sorted them and we have insert them in X and X_test.
# X is the numpy array computed from the train dataset (this is why it has 10k rows)
# X_test is the numpy array computed from the test dataset (this is why it has 5k rows).

# We have stored the target variable in a numpy array as well, called y.

# At the end of the cell we have added a sanity check, that prints the shape of X, X_test and y
# ============================================================

# Columns shared by both train and test (intersection of column names)
cols = sorted(set(train_df.columns) & set(test_df.columns))

# Feature columns are all common columns EXCEPT the id/target columns.
# These are the inputs for our model.
f_cols = [c for c in cols if c not in ("battle_id", "player_won")]

# Target vector: did player 1 win the battle? (0 = no, 1 = yes)
# We convert to int and then to a 1D numpy array.
y = train_df["player_won"].astype(int).to_numpy()

# Feature matrix for training:
#   - shape is (n_train_battles, n_features)
#   - all values are floats (numerical features only)
X = train_df[f_cols].to_numpy(dtype=float)

# Feature matrix for the test set (same columns, same order)
X_test = test_df[f_cols].to_numpy(dtype=float)

# Sanity check: shapes of training and test matrices
print(f"X_train shape: {X.shape},\ny_train shape: {y.shape}")
print(f"X_test  shape: {X_test.shape}")


X_train shape: (10000, 66),
y_train shape: (10000,)
X_test  shape: (5000, 66)


In [3]:
# ============================================================
# 3) LOGISTIC REGRESSION MODEL + GRID SEARCH + SUBMISSION
# ------------------------------------------------------------
# In this cell we:
#   1. Build a pipeline: StandardScaler + LogisticRegression.
#   2. Define a small hyperparameter grid around a good region.
#   3. Run GridSearchCV with 5-fold Stratified cross-validation.
#   4. Take the best model found by CV.
#   5. Fit it on ALL the training data.
#   6. Predict on X_test and save the submission CSV file.
#

# ============================================================

# --- Imports for the model and for cross-validation/search ---
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# 1) Define the pipeline.
#    A pipeline is like a small factory: it first transforms the data,
#    then applies the final estimator.
#    Here we:
#      - scale every numeric feature (StandardScaler)
#      - apply a LogisticRegression classifier on top of the scaled features
pipe = Pipeline([
    ("scale",  StandardScaler()),  # step 1: scale features to mean 0, std 1
    ("clf",    LogisticRegression(random_state=42, max_iter=12000))  # step 2: linear classifier
])

# 2) Hyperparameter grid for the Logistic Regression.
# The parameters are the following:
#    Parameters:
#      - solver:       'saga' (supports elasticnet penalty)
#      - penalty:      'elasticnet' (mixture of L1 and L2 regularization)
#      - C:            inverse of regularization strength
#                      (smaller C = stronger regularization)
#      - l1_ratio:     how much L1 vs L2 we want in the elasticnet
#      - tol:          numerical tolerance for the optimizer
#      - class_weight: class weights (we keep None here)
#      - n_jobs:       number of CPU cores for LogisticRegression
param_grid = {
    "clf__solver":      ["saga"],
    "clf__penalty":     ["elasticnet"],
    "clf__C":           [0.0089, 0.0092, 0.0098, 0.0101],
    "clf__l1_ratio":    [0.070, 0.073, 0.077, 0.080],
    "clf__tol":         [8e-05, 9e-05, 1.1e-04, 1.2e-04],
    "clf__class_weight":[None],
    "clf__n_jobs":      [-1],
}

# 3) Cross-validation splitter.
#    StratifiedKFold means each fold keeps roughly the same ratio of
#    wins/losses (0/1 in y) as the full dataset.
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4) Define the grid search object.
#    GridSearchCV will:
#      - try all combinations in param_grid
#      - for each combination, train/validate the model on the CV folds
#      - keep track of the mean CV accuracy for each set of parameters
#      - refit the best combination on ALL training data at the end
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="accuracy",  # we optimize for accuracy (fraction of correct predictions)
    cv=cv,
    n_jobs=-1,           # use all CPU cores available
    refit=True,          # after the search, fit the best model on the full dataset
    verbose=1            # print progress information during the search
)

# Actually run the grid search on the training data (X, y).
grid.fit(X, y)

# Print the best score (mean accuracy across CV folds) and the hyperparameters
print("best score (CV mean accuracy):", round(grid.best_score_, 4))
print("best hyperparameters:")
for k, v in grid.best_params_.items():
    print(f"  - {k}: {v}")

# This is the final pipeline (scaler + logistic regression) trained
# with the best combination of hyperparameters found by GridSearchCV.
best_model = grid.best_estimator_

# 5) Submission: train on ALL training data and predict on X_test.
#    Note: GridSearchCV with refit=True already fits best_model on all
#    training data, but calling fit(X, y) again here does not change
#    the final result; it just makes the step explicit.
best_model.fit(X, y)
test_pred = best_model.predict(X_test).astype(int)

# Build the submission DataFrame:
#   - battle_id  : copied directly from test_df
#   - player_won : prediction of our model (0 or 1) for each battle
submission = pd.DataFrame({
    "battle_id": test_df["battle_id"],
    "player_won": test_pred
})


# SUBMISSION
# Here we are building the submission DataFrame for the competition/evaluation.
submission.to_csv("submission1.csv", index=False)
print("\nSaved: submission_lr.csv")


Fitting 5 folds for each of 64 candidates, totalling 320 fits
best score (CV mean accuracy): 0.8512
best hyperparameters:
  - clf__C: 0.0092
  - clf__class_weight: None
  - clf__l1_ratio: 0.077
  - clf__n_jobs: -1
  - clf__penalty: elasticnet
  - clf__solver: saga
  - clf__tol: 8e-05

Saved: submission_lr.csv
